# Day 14 Processing a raster by block

As read and coded from chpt 9 in Geoprocessing with Python by Chris Garrard 

0. Libraries required

In [1]:
from pathlib import Path
import os
from osgeo import gdal
import numpy as np
from sentinelsat import SentinelAPI

1. Set up all paths required in script

In [2]:
path = Path('../data/rivers_amazon_data/ndwi_test/')

2. Download the Sentinel image, extract bands as tifs and store seperately.

The uuid is taken from https://scihub.copernicus.eu/dhus/#/home

In [ ]:
# username = os.environ.get('COPERNICUS_USERNAME')
# password = os.environ.get('COPERNICUS_PASSWORD')
# api = SentinelAPI(user=username, password=password)
# api.download('dcab558f-9b8d-4935-99d5-881144e17348', path)
# product_id = (str(list(path.glob('*'))).split('/')[-1])[:-7];
# ! unzip {path}/{product_id + '.zip'} -d {path}

# parts = [product_id + '.SAFE', 
#         'GRANULE',
#         'L1C_T21MYS_A016080_20180721T140053',
#         'IMG_DATA']

# for i in [2,3,4,8]:
#     image_band = 'T21MYS_20180721T140051_B0{}.jp2'.format(i)
#     filepath = str(path.joinpath(*parts).joinpath(image_band))
#     out_location = str(path) + ('/ndwi_B0{}'.format(i)) + '.tif'
#     src = gdal.Open(filepath)
#     gdal.Translate(out_location, src)

3. Create a False color image.

Now that I have the 4 10m bands for S2 imagery stack them

In [3]:
band1_fn = str(path.joinpath('ndwi_B08.tif')) #NIR
band2_fn = str(path.joinpath('ndwi_B02.tif')) #Green
band3_fn = str(path.joinpath('ndwi_B03.tif')) #Blue

in_ds = gdal.Open(band1_fn)
in_band = in_ds.GetRasterBand(1)
gtiff_driver = gdal.GetDriverByName('GTiff')
out_path = str(path) + '/false_color.tif'
out_ds = gtiff_driver.Create(out_path, in_band.XSize,
                            in_band.YSize, 3, in_band.DataType)
out_ds.SetProjection(in_ds.GetProjection())
out_ds.SetGeoTransform(in_ds.GetGeoTransform())

in_data = in_band.ReadAsArray()
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(in_data)

out_ds.GetRasterBand(2).WriteArray(gdal.Open(band2_fn).ReadAsArray())
out_ds.GetRasterBand(3).WriteArray(gdal.Open(band3_fn).ReadAsArray())

out_ds.FlushCache()
for i in range(1,4):
    out_ds.GetRasterBand(i).ComputeStatistics(False)
out_ds.BuildOverviews('average', [2,3,8,16,32])

del out_ds

NDWI, defined as (GREEN-NIR)/(GREEN+NIR)

Collect information about the image dataset and assign as variables

In [ ]:
in_ds = gdal.Open(str(input_im))

In [ ]:
in_band = in_ds.GetRasterBand(1)
xsize = in_band.XSize
ysize = in_band.YSize
block_xsize, block_ysize = in_band.GetBlockSize()
nodata = in_band.GetNoDataValue()

In [ ]:
print('Image size is', xsize, 'by', ysize, '\n', 
      'Block size is', block_xsize, 'by', block_ysize)

Create an output dataset as an empty dataframe

In [ ]:
gtiff_driver = gdal.GetDriverByName('GTiff')
out_ds = gtiff_driver.Create('ndwi_test.tif', xsize, ysize,
                                 1, in_band.DataType)

In [ ]:
out_ds.SetProjection(in_ds.GetProjection())
out_ds.SetGeoTransform(in_ds.GetGeoTransform())
out_band = out_ds.GetRasterBand(1)

In [ ]:
! unzip {datapath}/{product_df.iloc[0][0] + '.zip'} -d {datapath}